In [1]:
%load_ext autoreload
%autoreload

import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer, StandardScaler
sys.path.append('../../utils')
from utils import time_comparison

### Time fiability


In [7]:
gt = pd.read_csv("../../../dumps/time_analysis/threshold_3/3_20190615_6000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_train = gt[cols]
target_train = gt['label']

scaler = StandardScaler()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

tree = LogisticRegression(C=10, max_iter=1000,random_state=0)
tree.fit(data_train, target_train)
dump(tree,"snapshots/logreg_3_20190615_6000.joblib")

['snapshots/logreg_3_20190615_6000.joblib']

In [8]:
tree = load("snapshots/logreg_3_20190615_6000.joblib")

gt = pd.read_csv("../../../dumps/time_analysis/threshold_3/3_20190808_1000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_test = gt[cols]
target_test = gt['label']

data_test = scaler.transform(data_test)

print("Accuracy on training set: {:.3f}".format(tree.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(tree.score(data_test, target_test)))

Accuracy on training set: 0.998
Accuracy on test set: 0.985


#### Long run

In [2]:
time_comparison('log','../../')

Acceptation threshold : 1/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.968          0.752
                       14000                   2               0.969143       0.759
                       21000                   3               0.95081        0.71
                       31000                   4.42857         0.895839       0.709
Acceptation threshold : 2/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.99           0.986
                       14000                   2               0.988143       0.986
                       21000                   3               0.972667       0.986
                 

Increasing the size of the training set does improve the accuracies but not significantly.
Performances over time are correct but clearly degraded.